In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import date
import math

In [141]:
vwce = pd.DataFrame(yf.download('VWCE.MI'))
liquidity= pd.DataFrame(yf.download('C3M.MI'))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [142]:
def preprocessing(df:pd.DataFrame):
    df.columns = df.columns.droplevel('Ticker')
    df.reset_index(inplace=True)
    df.rename(columns={"Price": "Index"}, inplace=True)
    new_df=pd.DataFrame(columns=["Date","Close","High","Low","DD%"])
    new_df["Date"]=df["Date"]
    new_df["Date"]=new_df['Date'].dt.date
    new_df["Close"]=df["Close"]
    new_df["High"]=df["High"]
    new_df["Low"]=df["Low"]
    #Calcolo il drawdown
    maximum = 0
    for i in range(0,len(new_df)):
        if new_df.loc[i,"High"] >= maximum:
            maximum = new_df.loc[i,"High"]
            dd = 0.0
            new_df.loc[i,"DD%"] = dd
        else:
            dd = ((new_df.loc[i,"Close"] - maximum) / maximum)*100
            new_df.loc[i,"DD%"] = dd.round(2)
    return new_df

In [143]:
vwce=preprocessing(vwce)
liquidity=preprocessing(liquidity)

In [144]:
vwce
filtered=vwce.loc[vwce['DD%'] < -10]
print(filtered)

           Date      Close       High        Low    DD%
32   2020-02-27  73.449997  74.860001  72.589996 -10.43
33   2020-02-28  70.239998  71.029999  69.680000 -14.34
34   2020-03-02  71.320000  72.169998  70.029999 -13.02
35   2020-03-03  71.879997  73.379997  72.309998 -12.34
36   2020-03-04  72.599998  73.050003  71.879997 -11.46
..          ...        ...        ...        ...    ...
840  2023-04-27  94.519997  94.540001  93.879997 -10.69
842  2023-05-02  94.379997  95.980003  94.349998 -10.82
843  2023-05-03  94.440002  94.959999  94.400002 -10.76
844  2023-05-04  94.059998  94.260002  93.639999 -11.12
845  2023-05-05  95.220001  95.320000  94.199997 -10.03

[272 rows x 5 columns]


In [145]:
liquidity

,Date,Close,High,Low,DD%
0,2010-10-12,118.320000,118.320000,118.320000,0.0
1,2010-10-18,118.320000,118.320000,118.320000,0.0
2,2010-10-21,118.320000,118.320000,118.320000,0.0
3,2010-10-26,118.320000,118.320000,118.320000,0.0
4,2010-10-27,118.320000,118.320000,118.320000,0.0
...,...,...,...,...,...
3464,2024-12-05,122.660004,122.709999,122.570000,-0.07
3465,2024-12-06,122.730003,122.760002,122.660004,0.0
3466,2024-12-09,122.760002,122.849998,122.669998,0.0
3467,2024-12-10,122.660004,122.839996,122.660004,-0.15


In [146]:
#Defining needed variables
thresholds = np.arange(0, -55, -5)
buffer_percentages = np.array([0.0 ,0.30, 0.20, 0.10, 0.05, 0.05, 0.20, 0.30, 0.40, 0.50, 0.50])
thresholds_dict = dict(zip(thresholds, buffer_percentages))

print(thresholds)

[  0  -5 -10 -15 -20 -25 -30 -35 -40 -45 -50]


In [ ]:
def simulation_padd(etf_df,liquidity_df,start_date,end_date):
    last_threshold_alert=0
    monthly_investment=500
    number_of_stocks=0
    number_of_liquidity=0
    buffer_value=0
    pmc_etf=0
    pmc_liquidity=0
    #Set start and end date
    start_date=max(start_date,etf_df["Date"].iloc[0],liquidity_df["Date"].iloc[0])
    end_date=min(end_date,etf_df["Date"].iloc[-1],liquidity_df["Date"].iloc[-1])
    print("Start date: "+str(start_date))
    print("End date: "+str(end_date))
    #Filter dataframes by start and end date
    etf_df = etf_df[(etf_df['Date'] >= start_date) & (etf_df['Date'] <= end_date)]
    liquidity_df = liquidity_df[(liquidity_df['Date'] >= start_date) & (liquidity_df['Date'] <= end_date)]
    #Reset index
    etf_df.reset_index(drop=True, inplace=True)
    liquidity_df.reset_index(drop=True, inplace=True)
    
    for i in range(0,min(len(etf_df),len(liquidity_df))):
        number_of_purchased_stocks=0
        number_of_purchased_liquidity=0
        if etf_df.loc[i,"Date"] < start_date or etf_df.loc[i,"Date"] > end_date or liquidity_df.loc[i,"Date"] < start_date or liquidity_df.loc[i,"Date"] > end_date:
            continue
        #Controllo se ho raggiunto il massimo e nel caso resetto l'ultimo alert
        if etf_df.loc[i,"DD%"] == 0:
            last_threshold_alert = 0
            print("MAXIMUM REACHED")
        #All'inizio del mese acquisto un'azione, aggiorno il PMC e il buffer
        if etf_df.iloc[i]["Date"].month != etf_df.iloc[i-1]["Date"].month:
            pmc_etf = ((pmc_etf * number_of_stocks) + (etf_df.loc[i,"Close"] * 1))/(number_of_stocks+1)
            number_of_stocks+=1
            if(i>30 and liquidity_df.loc[i,"Close"] - liquidity_df.loc[i-30,"Close"] < 0):
                buffer_value = buffer_value + monthly_investment - etf_df.loc[i,"Close"]*1
            else:
                number_of_purchased_liquidity = int((buffer_value + monthly_investment - (etf_df.loc[i,"Close"] * 1))  / liquidity_df.loc[i,"Close"])
                number_of_purchased_liquidity = int((buffer_value + monthly_investment - (etf_df.loc[i,"Close"] * 1))  / liquidity_df.loc[i,"Close"])
                pmc_liquidity = ((pmc_liquidity * number_of_liquidity) + (liquidity_df.loc[i,"Close"] * number_of_purchased_liquidity))/(number_of_liquidity+number_of_purchased_liquidity)
                buffer_value = buffer_value + (monthly_investment - etf_df.loc[i,"Close"]*1 - liquidity_df.loc[i,"Close"]*number_of_purchased_liquidity)
                number_of_liquidity+=number_of_purchased_liquidity
            print("PMC_ETF: "+str(pmc_etf))
            print("Numero di azioni: "+str(number_of_stocks))
            print("PMC_LIQUIDITY: "+str(pmc_liquidity))
            print("Numero di azioni di liquidità: "+str(number_of_liquidity))
            print("Buffer: "+str(buffer_value))
        #Se il drawdown è maggiore di una delle soglie investi con una percentuale del buffer
        for j in range(len(thresholds) - 1):
            if thresholds[j] >= etf_df.loc[i,"DD%"] > thresholds[j + 1]:
                print(f"{etf_df.loc[i,'DD%']} è compreso tra {thresholds[j + 1]} e {thresholds[j]}")
                break
        if thresholds[j] < last_threshold_alert and thresholds[j] != 0 and buffer_value > 0:
            print("### ALERT ### "+str(thresholds[j]))
            last_threshold_alert = thresholds[j]
            #Alert scattato: investo una percentuale del buffer
            print("### ACQUISTO ###")
            ### AAA CAMBIARE LOGICA: AUMENTARE L'INVESTIMENTO IN BASE AI MESI TRASCORSI
            total_buffer=buffer_value+pmc_liquidity*number_of_liquidity
            print("Total buffer before purchase: "+str(total_buffer))
            print("Total number of liquidity before purchase: "+str(number_of_liquidity))
            print("PMC ETF before purchase: "+str(pmc_etf))
            to_be_invested = total_buffer * thresholds_dict[thresholds[j]]
            if(to_be_invested > buffer_value):
                number_of_purchased_stocks = int((to_be_invested) / etf_df.loc[i,"Close"])
                number_of_purchased_stocks_with_buffer = int(buffer_value / etf_df.loc[i,"Close"])
                number_of_purchased_stocks_with_liquidity = number_of_purchased_stocks - number_of_purchased_stocks_with_buffer
                print("Numero di azioni acquistate: "+str(number_of_purchased_stocks))
                print("Numero di azioni acquistate con buffer: "+str(number_of_purchased_stocks_with_buffer))
                print("Numero di azioni acquistate liquidando: "+str(number_of_purchased_stocks_with_liquidity))
                number_of_liquidity_to_be_sold = math.ceil((etf_df.loc[i,"Close"] * number_of_purchased_stocks_with_liquidity) / pmc_liquidity)
                number_of_liquidity-=number_of_liquidity_to_be_sold
                buffer_value = buffer_value + number_of_liquidity_to_be_sold * pmc_liquidity
                buffer_value = buffer_value - (number_of_purchased_stocks * etf_df.loc[i,"Close"])
                number_of_stocks += number_of_purchased_stocks
                pmc_etf = ((pmc_etf * number_of_stocks) + (etf_df.loc[i,"Close"] * number_of_purchased_stocks))/(number_of_stocks+number_of_purchased_stocks)
                print("PMC ETF updated: "+str(pmc_etf))
                print("Total buffer updated: "+str(total_buffer))
                print("Cash buffer updated: "+str(buffer_value))
                print("Total number of liquidity updated: "+str(number_of_liquidity))
                print("######################")
            else:    
                number_of_purchased_stocks = int(buffer_value * thresholds_dict[thresholds[j]] / etf_df.loc[i,"Close"])
                print("Numero di azioni acquistate: "+str(number_of_purchased_stocks))
                buffer_value = buffer_value - (number_of_purchased_stocks * etf_df.loc[i,"Close"])
                print("Buffer updated: "+str(buffer_value))
                number_of_stocks += number_of_purchased_stocks
                pmc_etf = ((pmc_etf * number_of_stocks) + (etf_df.loc[i,"Close"] * number_of_purchased_stocks))/(number_of_stocks+number_of_purchased_stocks)
                print("PMC ETF updated: "+str(pmc_etf))
                print("######################")
        
            ### AAA METTERE RENDIMENTO 1.5 DEL BUFFER 
            ### AAA non investire se il rendimento è negativo per LIQUIDITY
            ### AAA Refactoring!!!!!!!!
            ### AAA aggiungere un dizionario così formato {date: (pmc_etf,number_of_stocks,pmc_liquidity,number_of_liquidity,buffer_value)} in modo tale da calcolare rendimenti nel tempo e deviazione std
            ### AAA RIAZZERARE ALERT SE SONO PASSATI PIU DI X MESI DALL'ULTIMO ALERT 
            ### AAA la threshold sempre decrescente (<) o anche in risalita (!=)? if thresholds[i] != last_threshold_alert and thresholds[i] != 0 and buffer_value > 0


In [148]:
simulation_padd(vwce,liquidity,date(2015, 12, 12),date(2025, 12, 12))

Start date: 2020-01-14
End date: 2024-12-12
MAXIMUM REACHED
PMC_ETF: 78.62000274658203
Numero di azioni: 1
PMC_LIQUIDITY: 117.41000366210938
Numero di azioni di liquidità: 3
Buffer: 69.14998626708984
0.0 è compreso tra -5 e 0
-0.23 è compreso tra -5 e 0
MAXIMUM REACHED
0.0 è compreso tra -5 e 0
MAXIMUM REACHED
0.0 è compreso tra -5 e 0
-0.16 è compreso tra -5 e 0
-0.51 è compreso tra -5 e 0
MAXIMUM REACHED
0.0 è compreso tra -5 e 0
-0.81 è compreso tra -5 e 0
-0.81 è compreso tra -5 e 0
-2.69 è compreso tra -5 e 0
-1.6 è compreso tra -5 e 0
-1.6 è compreso tra -5 e 0
-2.57 è compreso tra -5 e 0
-3.71 è compreso tra -5 e 0
PMC_ETF: 78.04500198364258
Numero di azioni: 2
PMC_LIQUIDITY: 117.41000366210938
Numero di azioni di liquidità: 7
Buffer: 22.03997039794922
-2.9 è compreso tra -5 e 0
-1.27 è compreso tra -5 e 0
MAXIMUM REACHED
0.0 è compreso tra -5 e 0
MAXIMUM REACHED
0.0 è compreso tra -5 e 0
-0.4 è compreso tra -5 e 0
-0.09 è compreso tra -5 e 0
MAXIMUM REACHED
0.0 è compreso tra -

In [149]:
def simulation_pac(etf_df):
    pmc = 0
    number_of_stocks = 0
    saved_money=0
    global monthly_investment
    for i in range(0,len(etf_df)):
        #All'inizio del mese acquisto tot azioni quante posso permettermene, aggiorno il PMC e il buffer
        if etf_df.iloc[i]["Date"].month != etf_df.iloc[i-1]["Date"].month:
            number_of_stocks_purchased = int((monthly_investment+saved_money) / etf_df.loc[i,"Close"])
            saved_money = monthly_investment + saved_money - (number_of_stocks_purchased * etf_df.loc[i,"Close"])
            pmc = ((pmc * number_of_stocks) + (etf_df.loc[i,"Close"] * number_of_stocks_purchased))/(number_of_stocks+number_of_stocks_purchased)
            number_of_stocks+=number_of_stocks_purchased
            print("PMC: "+str(pmc))
            print("Numero di azioni: "+str(number_of_stocks))
            print("Saved Money: "+str(saved_money))

In [150]:
simulation_pac(vwce)

PMC: 78.62000274658203
Numero di azioni: 6
Saved Money: 28.279983520507812
PMC: 78.04500198364258
Numero di azioni: 12
Saved Money: 63.45997619628906
PMC: 75.56736956144634
Numero di azioni: 19
Saved Money: 64.21997833251953
PMC: 70.55714348384312
Numero di azioni: 28
Saved Money: 24.399982452392578
PMC: 69.67771551949637
Numero di azioni: 35
Saved Money: 61.27995681762695
PMC: 69.87309601193382
Numero di azioni: 42
Saved Money: 65.3299674987793
PMC: 70.13979627648179
Numero di azioni: 49
Saved Money: 63.14998245239258
PMC: 70.49232162748065
Numero di azioni: 56
Saved Money: 52.429988861083984
PMC: 71.05984127710737
Numero di azioni: 63
Saved Money: 23.229999542236328
PMC: 71.38681157430013
Numero di azioni: 69
Saved Money: 74.31000137329102
PMC: 71.63118427678158
Numero di azioni: 76
Saved Money: 56.02999496459961
PMC: 72.27426822011064
Numero di azioni: 82
Saved Money: 73.51000595092773
PMC: 72.97393237338977
Numero di azioni: 89
Saved Money: 5.320018768310547
PMC: 73.6273682042172
N